# Отток клиентов

# Описание задачи
Из банка стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Нам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком.

Построим модель с предельно большим значением F1-меры. Нужно довести метрику до 0.59. Проверим F1-меру на тестовой выборке.
Дополнительно измерим AUC-ROC, сравнив её значение с F1-мерой.

# План работы
- Загрузка данных и их подготовка
- Исследование задачи
- Борьба с дисбалансом
- Тестирование модели
- Вывод

# Описание данных:

**Признаки:**

- RowNumber: индекс строки в данных
- CustomerId: уникальный идентификатор клиента
- Surname: фамилия
- CreditScore: кредитный рейтинг
- Geography: страна проживания
- Gender: пол
- Age: возраст
- Tenure: сколько лет человек является клиентом банка
- Balance: баланс на счёте
- NumOfProducts: количество продуктов банка, используемых клиентом
- HasCrCard: наличие кредитной карты
- IsActiveMember: активность клиента
- EstimatedSalary: предполагаемая зарплата

**Целевой признак:**

- Exited: факт ухода клиента

# Загрузка данных и их подготовка

In [ ]:
! pip install phik

     |████████████████████████████████| 679 kB 2.1 MB/s eta 0:00:01


In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from collections import Counter
from sklearn.datasets import make_classification

In [ ]:
# чтение файла данных в датафрейм из папки по умолчанию и из рабочей директории
try:
    df = pd.read_csv('/datasets/Churn.csv', sep=',')
except:
    df = pd.read_csv('/content/Churn.csv', sep=',')

In [ ]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


Признаки
- `RowNumber` — индекс строки в данных
- `CustomerId` — уникальный идентификатор клиента
- `Surname` — фамилия
- `CreditScore` — кредитный рейтинг
- `Geography` — страна проживания
- `Gender` — пол
- `Age` — возраст
- `Tenure` — сколько лет человек является клиентом банка
- `Balance` — баланс на счёте
- `NumOfProducts` — количество продуктов банка, используемых клиентом
- `HasCrCard` — наличие кредитной карты
- `IsActiveMember` — активность клиента
- `EstimatedSalary` — предполагаемая зарплата  

Целевой переменная
- `Exited` — факт ухода клиента

In [ ]:
df.isna().mean()

RowNumber          0.0000
CustomerId         0.0000
Surname            0.0000
CreditScore        0.0000
Geography          0.0000
Gender             0.0000
Age                0.0000
Tenure             0.0909
Balance            0.0000
NumOfProducts      0.0000
HasCrCard          0.0000
IsActiveMember     0.0000
EstimatedSalary    0.0000
Exited             0.0000
dtype: float64

Пропуски в столбце годы клиента в банке Tenure = 0.0909

In [ ]:
df.describe(include='all')

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000,10000.000000,10000,10000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
unique,NaN,NaN,2932,NaN,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,Smith,NaN,France,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,32,NaN,5014,5457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,5000.50000,1.569094e+07,NaN,650.528800,NaN,NaN,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,NaN,96.653299,NaN,NaN,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,NaN,350.000000,NaN,NaN,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,NaN,584.000000,NaN,NaN,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,NaN,652.000000,NaN,NaN,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,NaN,718.000000,NaN,NaN,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000


In [ ]:
df['Exited'].value_counts(normalize=True)

0    0.7963
1    0.2037
Name: Exited, dtype: float64

Доля уходящих клиентов Exited = 0.2037

In [ ]:
df['Exited'].mean()

0.2037

Дисбаланс классов 0.2037 он достаточно сильный, чтобы негативно повлиять на качество моделей.

In [ ]:
df.corr()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,1.000000,0.004202,0.005840,0.000783,-0.007322,-0.009067,0.007246,0.000599,0.012044,-0.005988,-0.016571
CustomerId,0.004202,1.000000,0.005308,0.009497,-0.021418,-0.012419,0.016972,-0.014025,0.001665,0.015271,-0.006248
CreditScore,0.005840,0.005308,1.000000,-0.003965,-0.000062,0.006268,0.012238,-0.005458,0.025651,-0.001384,-0.027094
Age,0.000783,0.009497,-0.003965,1.000000,-0.013134,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.285323
Tenure,-0.007322,-0.021418,-0.000062,-0.013134,1.000000,-0.007911,0.011979,0.027232,-0.032178,0.010520,-0.016761
Balance,-0.009067,-0.012419,0.006268,0.028308,-0.007911,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.118533
NumOfProducts,0.007246,0.016972,0.012238,-0.030680,0.011979,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.047820
HasCrCard,0.000599,-0.014025,-0.005458,-0.011721,0.027232,-0.014858,0.003183,1.000000,-0.011866,-0.009933,-0.007138
IsActiveMember,0.012044,0.001665,0.025651,0.085472,-0.032178,-0.010084,0.009612,-0.011866,1.000000,-0.011421,-0.156128
EstimatedSalary,-0.005988,0.015271,-0.001384,-0.007201,0.010520,0.012797,0.014204,-0.009933,-0.011421,1.000000,0.012097


с помощью кореляции посмотрим признаки  на предмет мультиколлинеарности - сильной линейной зависимости между нецелевыми признаками не обнаружилось.

In [ ]:
df.columns = df.columns.str.replace(r"([A-Z])", r" \1").str.lower().str.replace(' ', '_').str[1:]

исправим названия столбцов

In [ ]:
df

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


т.к. столбцы с номерами и фамилии не представляют интереса для исследования, tenure есть немного пропусков удалим их

In [ ]:
df = df.drop(['row_number', 'surname', 'customer_id'], axis=1)

заполним пропуски.

In [ ]:
data_products = df.groupby('age')['tenure'].transform('median')
df['tenure'] = df['tenure'].fillna(data_products) #присваиваем медиальное значение в количества продуктов

In [ ]:
df

,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5.0,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10.0,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7.0,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3.0,75075.31,2,1,0,92888.52,1


In [ ]:
# произведем кодирование
df = pd.get_dummies(df, drop_first=True)

In [ ]:
df

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,geography_Germany,geography_Spain,gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5.0,0.00,2,1,0,96270.64,0,0,0,1
9996,516,35,10.0,57369.61,1,1,1,101699.77,0,0,0,1
9997,709,36,7.0,0.00,1,0,1,42085.58,1,0,0,0
9998,772,42,3.0,75075.31,2,1,0,92888.52,1,1,0,1


In [ ]:
# выделение обучающей и тестовой выборкок \random_state=42\
train, test = train_test_split(df,train_size=0.6,random_state=1432, stratify=df['exited'])
valid, test = train_test_split(test,train_size=0.5,random_state=1432, stratify=test['exited'])

In [ ]:
features_train = train.drop(['exited'], axis=1)
target_train = train['exited']
features_valid = valid.drop(['exited'], axis=1)
target_valid = valid['exited']
features_test = test.drop(['exited'], axis=1)
target_test = test['exited']

In [ ]:
# произведем масштабирование
numeric = ['credit_score', 'age', 'tenure', 'balance', 'estimated_salary', 'num_of_products']
scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

# Исследование задачи

In [ ]:
rf_model = RandomForestClassifier(n_estimators=7, random_state=14, max_depth=5)
rf_model.fit(features_train, target_train) # обучим модель

predictions_valid = rf_model.predict(features_valid) # найдём предсказания на валидационной выборке
print("Random Forest accuracy: =", accuracy_score(target_valid, predictions_valid))
print("Random Forest F1: =", f1_score(predictions_valid, target_valid))
probabilities = rf_model.predict_proba(features_train)
probabilities_one = probabilities[:, 1]
print('Площадь ROC-кривой:', roc_auc_score(target_train, probabilities_one))

Random Forest accuracy: = 0.85
Random Forest F1: = 0.49832775919732436
Площадь ROC-кривой: 0.8514872105442362


In [ ]:
dt_best_model = DecisionTreeClassifier(random_state=14, max_depth=5)
dt_best_model.fit(features_train, target_train) # обучим модель

predictions_valid = dt_best_model.predict(features_valid) # найдём предсказания на валидационной выборке
print("Random Forest accuracy: =", accuracy_score(target_valid, predictions_valid))
print("Random Forest F1: =", f1_score(predictions_valid, target_valid))
print("Random Forest roc_auc: =", roc_auc_score(predictions_valid, target_valid))

Random Forest accuracy: = 0.85
Random Forest F1: = 0.5176848874598071
Random Forest roc_auc: = 0.8070192881288527


In [ ]:
%%time
# подбор гиперпараметров случайного леса
rf_best_model = None
rf_best_result = 0
# списки для построения графика
rf_best_depth = 0
score_depth = [1, 12]
est_score_train = []
est_score_val = []
# подбор глубины дерева
for depth in range(1, 12):
    score_train = []
    score_val = []
    #подборка деревьев
    for est in range(1,100):
        model = RandomForestClassifier(n_estimators=est, random_state=14, max_depth=depth) # создадим модель, указав max_depth=depth
        model.fit(features_train, target_train) # обучим модель
        predictions_valid = model.predict(features_valid)
        score_train.append(model.score(features_train, target_train))
        #расчет точности на вал выборке
        result = f1_score(predictions_valid, target_valid)
        score_val.append(result)
        score_depth.append(depth)
        # определение лучшей модели
        if result > rf_best_result:
            rf_best_model = model
            rf_best_result = result
            rf_best_depth = depth
    #запись списков для лучшей глубины дерева
    if depth == rf_best_depth:
        est_score_train = score_train.copy()
        est_score_val = score_val.copy()
print("F1 наилучшей модели случайного леса: =", rf_best_result)
print("roc_auc наилучшей модели случайного леса: =", roc_auc_score(predictions_valid, target_valid))
print(rf_best_model)

F1 наилучшей модели случайного леса: = 0.5683563748079877
roc_auc наилучшей модели случайного леса: = 0.8071766360383861
RandomForestClassifier(max_depth=11, n_estimators=38, random_state=14)
CPU times: user 3min 55s, sys: 1.27 s, total: 3min 56s
Wall time: 3min 56s


In [ ]:
%%time

dt_best_model = None # подбор гиперпараметров решающего дерева
dt_best_result = 0

dt_best_depth = 0 # списки для построения графика
score_depth = [1, 16]
est_score_train = []
est_score_val = []
for depth in range(1, 100): # подбор глубины дерева
    score_train = []
    score_val = []
    model = DecisionTreeClassifier(random_state=14, max_depth=depth) # создадим модель, указав max_depth=depth
    model.fit(features_train, target_train) # обучим модель
    predictions_valid = model.predict(features_valid)
    score_train.append(model.score(features_train, target_train))
    result = f1_score(predictions_valid, target_valid) #расчет точности на вал выборке
    score_val.append(result)
    score_depth.append(depth)
    if result > dt_best_result: # определение лучшей модели
        dt_best_model = model
        dt_best_result = result
        dt_best_depth = depth
if depth == dt_best_depth: #запись списков для лучшей глубины дерева
    est_score_train = score_train.copy()
    est_score_val = score_val.copy()

print("F1 наилучшей модели случайного леса: =", dt_best_result)
print("roc_auc наилучшей модели случайного леса: =", roc_auc_score(predictions_valid, target_valid))
print(dt_best_model)

F1 наилучшей модели случайного леса: = 0.5470332850940666
roc_auc наилучшей модели случайного леса: = 0.6675758160419554
DecisionTreeClassifier(max_depth=8, random_state=14)
CPU times: user 2.8 s, sys: 20 ms, total: 2.82 s
Wall time: 2.83 s


In [ ]:
rf_best_model = RandomForestClassifier(n_estimators=38, random_state=14, max_depth=11)
rf_best_model.fit(features_train, target_train) # обучим модель

predictions_valid = rf_best_model.predict(features_valid) # найдём предсказания на валидационной выборке
print("Random Forest accuracy: =", accuracy_score(target_valid, predictions_valid))
print("Random Forest F1: =", f1_score(predictions_valid, target_valid))
probabilities = rf_best_model.predict_proba(features_train)
probabilities_one = probabilities[:, 1]
print('Площадь ROC-кривой:', roc_auc_score(target_train, probabilities_one))

Random Forest accuracy: = 0.8595
Random Forest F1: = 0.5683563748079877
Площадь ROC-кривой: 0.9765963955088754


In [ ]:
dt_best_model = DecisionTreeClassifier(random_state=14, max_depth=8)
dt_best_model.fit(features_train, target_train) # обучим модель

predictions_valid = dt_best_model.predict(features_valid) # найдём предсказания на валидационной выборке
print("DecisionTreeClassifier accuracy: =", accuracy_score(target_valid, predictions_valid))
print("DecisionTreeClassifier F1: =", f1_score(predictions_valid, target_valid))
print("DecisionTreeClassifierroc_auc: =", roc_auc_score(predictions_valid, target_valid))

DecisionTreeClassifier accuracy: = 0.839
DecisionTreeClassifier F1: = 0.5222551928783383
DecisionTreeClassifierroc_auc: = 0.7639297204950091


Проверим на тестовой выборке

Изучение несбалансированных данных показало необходимость использования F1-меры в качестве целевой метрики, что позволяет поддерживать баланс полноты и точности при идентификации объектов целевого класса. В дальнейшем были исследованы две основные модели классификаторов - логистическая регрессия и случайный лес. Однако дисбаланс классов не позволил ни одной модели добиться требуемой в задании точности.

# Борьба с дисбалансом

Напишем фунцию для увеличения выборки, что бы  объекты редкого класса не  были такими редкими

In [ ]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=14)

    return features_upsampled, target_upsampled

In [ ]:
features_upsampled_train, target_upsampled_train = upsample(features_train, target_train, repeat=4)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=38, random_state=14, max_depth=11)
rf_best_model.fit(features_train, target_train) # обучим модель

predictions_valid = rf_best_model.predict(features_valid) # найдём предсказания на валидационной выборке
print("Random Forest accuracy: =", accuracy_score(target_valid, predictions_valid))
print("Random Forest F1: =", f1_score(predictions_valid, target_valid))
probabilities = rf_best_model.predict_proba(features_train)
probabilities_one = probabilities[:, 1]
print('Площадь ROC-кривой:', roc_auc_score(target_train, probabilities_one))

Random Forest accuracy: = 0.844
Random Forest F1: = 0.5185185185185185
Площадь ROC-кривой: 0.9314967537383219


In [ ]:
dt_best_model = DecisionTreeClassifier(random_state=14, max_depth=8)
dt_best_model.fit(features_train, target_train) # обучим модель

predictions_valid = dt_best_model.predict(features_valid) # найдём предсказания на валидационной выборке
print("DecisionTreeClassifier accuracy: =", accuracy_score(target_valid, predictions_valid))
print("DecisionTreeClassifier F1: =", f1_score(predictions_valid, target_valid))
print("DecisionTreeClassifierroc_auc: =", roc_auc_score(predictions_valid, target_valid))

DecisionTreeClassifier accuracy: = 0.839
DecisionTreeClassifier F1: = 0.5222551928783383
DecisionTreeClassifierroc_auc: = 0.7639297204950091


In [ ]:
predictions_valid = rf_best_model.predict(features_valid) # найдём предсказания на валидационной выборке
print("Accuracy на валидационной выборке выборке: = " , accuracy_score(target_valid, predictions_valid))
print("F1 на валидационной выборке выборке: =", f1_score(predictions_valid, target_valid))
print("roc_auc на валидационной выборке выборке: =", roc_auc_score(predictions_valid, target_valid))

Accuracy на валидационной выборке выборке: =  0.844
F1 на валидационной выборке выборке: = 0.5185185185185185
roc_auc на валидационной выборке выборке: = 0.7818181818181819


Напишем фунцию для уменьшение выборки, что бы  объекты частого класса не были такими частыми

In [ ]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=14)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=14)] + [target_ones])

    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=14)

    return features_downsampled, target_downsampled

In [ ]:
features_downsampled_train, target_downsampled_train = downsample(features_train, target_train, fraction=0.25)

In [ ]:
predictions_valid = rf_best_model.predict(features_valid) # найдём предсказания на валидационной выборке
print("Accuracy на валидационной выборке выборке: = " , accuracy_score(target_test, predictions_valid))
print("F1 на валидационной выборке выборке: =", f1_score(predictions_valid, target_valid))
print("roc_auc на валидационной выборке выборке: =", roc_auc_score(predictions_valid, target_valid))

Accuracy на валидационной выборке выборке: =  0.728
F1 на валидационной выборке выборке: = 0.5683563748079877
roc_auc на валидационной выборке выборке: = 0.8171979922754602


In [ ]:
%%time
# подбор гиперпараметров случайного леса
rf_best_model = None
rf_best_result = 0
# списки для построения графика
rf_best_depth = 0
score_depth = [1, 16]
est_score_train = []
est_score_val = []
# подбор глубины дерева
for depth in range(1, 12):
    score_train = []
    score_val = []
    #подборка деревьев
    for est in range(1,75):
        model = RandomForestClassifier(n_estimators=est, random_state=14, max_depth=depth) # создадим модель, указав max_depth=depth
        model.fit(features_train, target_train) # обучим модель
        predictions_valid = model.predict(features_valid)
        score_train.append(model.score(features_train, target_train))
        #расчет точности на вал выборке
        result = f1_score(predictions_valid, target_valid)
        score_val.append(result)
        score_depth.append(depth)
        # определение лучшей модели
        if result > rf_best_result:
            rf_best_model = model
            rf_best_result = result
            rf_best_depth = depth
    #запись списков для лучшей глубины дерева
    if depth == rf_best_depth:
        est_score_train = score_train.copy()
        est_score_val = score_val.copy()
print("F1 наилучшей модели случайного леса: =", rf_best_result)
print("roc_auc наилучшей модели случайного леса: =", roc_auc_score(predictions_valid, target_valid))
print(rf_best_model)

F1 наилучшей модели случайного леса: = 0.5683563748079877
roc_auc наилучшей модели случайного леса: = 0.810171424823926
RandomForestClassifier(max_depth=11, n_estimators=38, random_state=14)
CPU times: user 2min 13s, sys: 492 ms, total: 2min 13s
Wall time: 2min 13s


In [ ]:
%%time

dt_best_model = None # подбор гиперпараметров решающего дерева
dt_best_result = 0

dt_best_depth = 0 # списки для построения графика
score_depth = [1, 16]
est_score_train = []
est_score_val = []
for depth in range(1, 12): # подбор глубины дерева
    score_train = []
    score_val = []
    model = DecisionTreeClassifier(random_state=14, max_depth=depth) # создадим модель, указав max_depth=depth
    model.fit(features_train, target_train) # обучим модель
    predictions_valid = model.predict(features_valid)
    score_train.append(model.score(features_train, target_train))
    result = f1_score(predictions_valid, target_valid) #расчет точности на вал выборке
    score_val.append(result)
    score_depth.append(depth)
    if result > dt_best_result: # определение лучшей модели
        dt_best_model = model
        dt_best_result = result
        dt_best_depth = depth
if depth == dt_best_depth: #запись списков для лучшей глубины дерева
    est_score_train = score_train.copy()
    est_score_val = score_val.copy()

print("F1 наилучшей модели случайного леса: =", dt_best_result)
print("roc_auc наилучшей модели случайного леса: =", roc_auc_score(predictions_valid, target_valid))
print(dt_best_model)

F1 наилучшей модели случайного леса: = 0.5470332850940666
roc_auc наилучшей модели случайного леса: = 0.7198899331882268
DecisionTreeClassifier(max_depth=8, random_state=14)
CPU times: user 192 ms, sys: 3.97 ms, total: 196 ms
Wall time: 198 ms


In [ ]:
rf_best_model = RandomForestClassifier(n_estimators=38, random_state=14, max_depth=11)

rf_best_model.fit(features_downsampled_train, target_downsampled_train) # обучим модель



predictions_valid = rf_best_model.predict(features_valid) # найдём предсказания на валидационной выборке
print("Random Forest accuracy: =", accuracy_score(target_valid, predictions_valid))
print("Random Forest F1: =", f1_score(predictions_valid, target_valid))
probabilities = rf_best_model.predict_proba(features_train)
probabilities_one = probabilities[:, 1]
print('Площадь ROC-кривой:', roc_auc_score(target_train, probabilities_one))

Random Forest accuracy: = 0.7745
Random Forest F1: = 0.5667627281460135
Площадь ROC-кривой: 0.9589185361987123


In [ ]:
dt_best_model = DecisionTreeClassifier(random_state=14, max_depth=7)
dt_best_model.fit(features_train, target_train) # обучим модель

predictions_valid = dt_best_model.predict(features_valid) # найдём предсказания на валидационной выборке
print("DecisionTreeClassifier accuracy: =", accuracy_score(target_valid, predictions_valid))
print("DecisionTreeClassifier F1: =", f1_score(predictions_valid, target_valid))
print("DecisionTreeClassifierroc_auc: =", roc_auc_score(predictions_valid, target_valid))

DecisionTreeClassifier accuracy: = 0.839
DecisionTreeClassifier F1: = 0.5222551928783383
DecisionTreeClassifierroc_auc: = 0.7639297204950091


Исправление дисбаланса классов в целевом признаке существенно повлияло на качество всех рассматриваемых моделей. Одна из них, построенная на классификаторе случайного леса показала качество прогноза, превышающее требуемый порог *F1*-меры в 0.59.  
Для исправления проблемы дисбаланса были использованы методы увеличения и уменьшение выборки.Эти два метода привели к улучшения качества модели.

В результате проделанной работы лучшей моделью являеться случайный лес. Апсемплинг помог улучшить показатели модели. Подобранные гиперпараметры глубина = 11, а колличество деревьев = 38


# Проверка на тестовой выборке

In [ ]:
predictions_test = rf_best_model.predict(features_test) # найдём предсказания на тестовой выборке
print("Accuracy на тестовой выборке выборке: = " , accuracy_score(target_test, predictions_test))
print("F1 на тестовой выборке выборке: =", f1_score(predictions_test, target_test))
probabilities = rf_best_model.predict_proba(features_train)
probabilities_one = probabilities[:, 1]
print('Площадь ROC-кривой:', roc_auc_score(target_train, probabilities_one))

Accuracy на тестовой выборке выборке: =  0.869
F1 на тестовой выборке выборке: = 0.6101190476190477
Площадь ROC-кривой: 0.9765963955088754


F1 = 0,61 целевой показатель достигут.

# Выводы


При обзоре данных было выявлено несколько неинформативных признаков: RowNumber, CustomerId, Surname. В столбце Tenure имеется не значительное количество пропусков. Несколько столбцов содержат категориальные признаки: Geography, Gender они были перекодированы.
Признаки были проверены на предмет мультиколлинеарности, но сильной линейной зависимости между нецелевыми признаками не обнаружилось. Целевой признак Exited не требует каких либо преобразований. В нем обнаружен значительный дисбаланс класс - целевой 1-й класс встречается почти в 4 раза реже 0-го класса. Такая неравномерность в распределении классов может негативно сказаться на качестве прогнозов.
Числовые признаки были подвергнуты масштабированию с помощью функции стандартизации StandardScaler, так как при анализе предполагается использовать чувствительные к масштабу данных классификаторы (напр. логистическую регрессию).

Изучение несбалансированных данных показало необходимость использования F1-меры в качестве целевой метрики, что позволяет поддерживать баланс полноты и точности при идентификации объектов целевого класса. В дальнейшем были исследованы две основные модели классификаторов - логистическая регрессия и случайный лес. Однако дисбаланс классов не позволил ни одной модели добиться требуемой в задании точности.
Исправление дисбаланса классов в целевом признаке существенно повлияло на качество всех рассматриваемых моделей. Одна из них, построенная на классификаторе случайного леса показала качество прогноза, превышающее требуемый порог *F1*-меры в 0.59.  
Для исправления проблемы дисбаланса были использованы методы увеличения и уменьшение выборки.Эти два метода привели к улучшения качества модели.
Были достигнуты цели поставленной задачи требуемый порог *F1*-меры в 0.59 был преодолён и проверен на тестовой выборке.  